In [5]:
# ✅ Install required packages (if not already installed)
# !pip install -q gradio pandas scikit-learn joblib requests python-docx openpyxl duckduckgo-search langchain langchain-core langchain-community langchain-groq python-dotenv

# ✅ Imports
import os, json, joblib, pandas as pd, numpy as np, requests
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from docx import Document
from openpyxl import Workbook
import gradio as gr
from duckduckgo_search import DDGS
from langchain_groq import ChatGroq
from dotenv import load_dotenv

# ✅ Load environment variables from .env file (optional but recommended)
load_dotenv()
groq_api = os.getenv("GROQ_API_KEY")

# ✅ Check for missing API key
if not groq_api:
    raise ValueError("❌ Missing GROQ_API_KEY. Please set it in your environment or .env file.")

# ✅ Load & train model (only once, comment after first run)
df = pd.read_csv(r"C:\Users\Admin\OneDrive\Desktop\heart.csv")  # Update path if needed
X = df.drop("target", axis=1)
y = df["target"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train_scaled, y_train)
joblib.dump(model, "heart_disease_model.pkl")
joblib.dump(scaler, "scaler.pkl")

# ✅ Load model and scaler
model = joblib.load("heart_disease_model.pkl")
scaler = joblib.load("scaler.pkl")

# ✅ LLM setup using Groq API
llm = ChatGroq(temperature=0, model_name="llama3-70b-8192", groq_api_key=groq_api)

REQUIRED_FEATURES = ["age", "sex", "cp", "trestbps", "chol", "fbs", "restecg",
                     "thalach", "exang", "oldpeak", "slope", "ca", "thal"]

# ✅ LLM Chat Function
def groq_llm_chat(system_msg, user_msg):
    url = "https://api.groq.com/openai/v1/chat/completions"
    headers = {"Authorization": f"Bearer {groq_api}", "Content-Type": "application/json"}
    data = {
        "model": "llama3-70b-8192",
        "messages": [{"role": "system", "content": system_msg}, {"role": "user", "content": user_msg}]
    }
    res = requests.post(url, headers=headers, json=data)
    return res.json()["choices"][0]["message"]["content"] if res.status_code == 200 else res.text

# ✅ Feature Extraction
def extract_features_llm(description):
    system_msg = (
        "You are a medical assistant. Extract only the required numeric features from a patient's case in valid JSON format. "
        "Keys: age, sex, cp, trestbps, chol, fbs, restecg, thalach, exang, oldpeak, slope, ca, thal. Reply only with valid JSON."
    )
    response = groq_llm_chat(system_msg, description)
    try:
        json_str = response[response.find("{"):response.rfind("}")+1]
        features = json.loads(json_str)
        for key in REQUIRED_FEATURES:
            if key not in features:
                raise ValueError(f"Missing {key}")
        return features
    except Exception as e:
        print("❌ Feature extraction failed:", e)
        return None

# ✅ Prediction
def predict_risk(features):
    X = pd.DataFrame([features])
    X_scaled = scaler.transform(X)
    return float(np.round(model.predict_proba(X_scaled)[0][1] * 100, 2))

# ✅ Lifestyle Advice
def lifestyle_advice():
    return (
        "- Maintain a heart-healthy diet\n"
        "- Exercise regularly (30 mins/day)\n"
        "- Avoid smoking and alcohol\n"
        "- Get enough sleep\n"
        "- Reduce stress and anxiety"
    )

# ✅ Reference Links
def get_reference_links(query):
    links = []
    try:
        refined_query = f"{query} heart disease site:mayoclinic.org OR site:wikipedia.org"
        with DDGS() as ddgs:
            for r in ddgs.text(refined_query, max_results=3):
                title, href = r.get("title", ""), r.get("href", "")
                links.append(f"{title}: {href}")
    except Exception as e:
        print("❌ Link error:", e)
    return "\n".join(links) if links else "No links found."

# ✅ Report Generators
def generate_docx_report(features, risk, analysis, advice, links):
    doc = Document()
    doc.add_heading("HEART DISEASE RISK REPORT", 0)
    doc.add_heading("Patient Features", level=1)
    for k, v in features.items():
        doc.add_paragraph(f"{k}: {v}")
    doc.add_heading("Risk Score", level=1)
    doc.add_paragraph(f"{risk}%")
    doc.add_heading("LLM Analysis", level=1)
    doc.add_paragraph(analysis)
    doc.add_heading("Lifestyle Advice", level=1)
    doc.add_paragraph(advice)
    doc.add_heading("Reference Links", level=1)
    doc.add_paragraph(links)
    path = "heart_report.docx"
    doc.save(path)
    return path

def generate_excel_report(features, risk, analysis, advice, links):
    wb = Workbook()
    ws = wb.active
    ws.append(["HEART DISEASE RISK REPORT"])
    ws.append(["Patient Features"])
    for k, v in features.items():
        ws.append([k, v])
    ws.append(["Risk Score", f"{risk}%"])
    ws.append(["LLM Analysis", analysis])
    ws.append(["Lifestyle Advice", advice])
    ws.append(["Reference Links", links])
    path = "heart_report.xlsx"
    wb.save(path)
    return path

# ✅ Main Processing Logic
def analyze(description, report_format):
    features = extract_features_llm(description)
    if not features:
        return "❌ Could not extract patient data. Try again.", None

    risk = predict_risk(features)
    advice = lifestyle_advice()
    analysis = groq_llm_chat(
        "You are a cardiologist. Explain the patient's heart disease risk briefly in formal language. Do not use emojis or brackets.",
        f"{features}, Risk: {risk}%"
    )
    links = get_reference_links(description)

    path = generate_excel_report(features, risk, analysis, advice, links) if report_format == "Excel" else generate_docx_report(features, risk, analysis, advice, links)

    summary = f"""🩺 **Risk Score**: {risk}%  
📌 **Lifestyle Advice**:\n{advice}\n\n🧠 **LLM Explanation**:\n{analysis[:500]}...\n\n🔗 **Reference Links**:\n{links}"""
    return summary, path

# ✅ Gradio UI
with gr.Blocks(theme=gr.themes.Base(primary_hue="red", secondary_hue="gray")) as demo:
    gr.Markdown("## 🫀 **Heart Disease Risk Assistant**")
    gr.Markdown("Quickly assess heart disease risk from natural language input. Get lifestyle advice, references, and downloadable report.")

    with gr.Row():
        with gr.Column(scale=3):
            with gr.Accordion("📋 Patient Input", open=True):
                description_input = gr.TextArea(
                    label="Describe the Patient's Condition",
                    placeholder="Example: A 66-year-old male with chest pain type 3, BP 180, cholesterol 320...",
                    lines=5,
                    max_lines=10
                )
                format_dropdown = gr.Radio(
                    choices=["DOCX", "Excel"],
                    label="Choose Report Format",
                    value="DOCX",
                    interactive=True
                )
                submit_btn = gr.Button("📊 Analyze & Generate Report", size="lg")

        with gr.Column(scale=4):
            with gr.Accordion("📄 Risk Summary & Download", open=True):
                result_summary = gr.Textbox(
                    label="🔎 Summary Output",
                    lines=15,
                    interactive=False,
                    show_copy_button=True
                )
                result_file = gr.File(label="⬇️ Download Your Report")

    submit_btn.click(fn=analyze, inputs=[description_input, format_dropdown], outputs=[result_summary, result_file])

demo.launch()


C:\Users\Admin\anaconda3\Lib\site-packages\IPython\utils\_process_win32.py:124: ResourceWarning: unclosed file <_io.BufferedWriter name=3>
  return process_handler(cmd, _system_body)
C:\Users\Admin\anaconda3\Lib\site-packages\IPython\utils\_process_win32.py:124: ResourceWarning: unclosed file <_io.BufferedReader name=4>
  return process_handler(cmd, _system_body)
C:\Users\Admin\anaconda3\Lib\site-packages\IPython\utils\_process_win32.py:124: ResourceWarning: unclosed file <_io.BufferedReader name=5>
  return process_handler(cmd, _system_body)
C:\Users\Admin\anaconda3\Lib\site-packages\gradio\http_server.py:120: ResourceWarning: unclosed <socket.socket fd=6224, family=2, type=1, proto=0>
  s = socket.socket()
C:\Users\Admin\anaconda3\Lib\site-packages\gradio\http_server.py:120: ResourceWarning: unclosed <socket.socket fd=6256, family=2, type=1, proto=0>
  s = socket.socket()
C:\Users\Admin\anaconda3\Lib\site-packages\gradio\http_server.py:120: ResourceWarning: unclosed <socket.socket fd

* Running on local URL:  http://127.0.0.1:7877
* To create a public link, set `share=True` in `launch()`.
